# ITSM - Forecasting and Predictions 

In [63]:
## Initiating Required Models
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder,scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,accuracy_score,classification_report,confusion_matrix
import pandas as pd
import numpy as np

In [2]:
## Required Database Credentials
Hostname = '13.250.191.87:3306'
username = 'dm_team'
userpwd = 'dm_team123#'
dbname = 'project_itsm'

In [3]:
## creating connection to database
conn = create_engine('mysql+pymysql://'+username+':'+userpwd+'@'+Hostname+'/'+dbname)

In [4]:
## Select query to the database
query = "select * from dataset_list"

In [5]:
## Extracting the data to a DataFrame
data = pd.read_sql(query,conn)

In [6]:
print(data.shape)
data.head()

(46606, 25)


,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4,0.601292279,...,,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1,SD0000007,2,,
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3,0.415049969,...,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1,SD0000011,1,,
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NA,0.517551335,...,,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1,SD0000017,,,
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4,0.642927218,...,,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1,SD0000025,,,
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4,0.345258343,...,,08-11-2013 13:55,08-11-2013 13:55,"3,38,39,03,333",Other,1,SD0000029,,,


In [7]:
print(data.columns)

Index(['CI_Name', 'CI_Cat', 'CI_Subcat', 'WBS', 'Incident_ID', 'Status',
       'Impact', 'Urgency', 'Priority', 'number_cnt', 'Category', 'KB_number',
       'Alert_Status', 'No_of_Reassignments', 'Open_Time', 'Reopen_Time',
       'Resolved_Time', 'Close_Time', 'Handle_Time_hrs', 'Closure_Code',
       'No_of_Related_Interactions', 'Related_Interaction',
       'No_of_Related_Incidents', 'No_of_Related_Changes', 'Related_Change'],
      dtype='object')


In [8]:
## Checking for any null values
data.isnull().values.any()

False

In [9]:
data.loc[data.Priority=='1'] ### 3 Prority 1 incidents

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
34301,SUB000282,subapplication,Server Based Application,WBS000295,IM0034604,Closed,1,1,1,0.014411679,...,12-02-2014 12:00,12-02-2014 12:01,12-02-2014 12:01,"3,02,66,11,111",Hardware,1,SD0104939,1,1,C00013982
35098,SUB000508,subapplication,Web Based Application,WBS000162,IM0035403,Closed,1,1,1,0.18281399,...,12-02-2014 12:02,12-02-2014 12:03,12-02-2014 12:03,"2,34,48,33,333",Software,1,SD0107440,26,,
36711,WBA000082,application,Web Based Application,WBS000055,IM0037024,Closed,1,1,1,0.653793843,...,17-02-2014 11:51,17-02-2014 11:54,17-02-2014 11:54,"16,975",Other,2,#MULTIVALUE,17,,


In [10]:
data.loc[data.Priority=='2'] ## 697 incidents

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
849,WSR000008,computer,Windows Server,WBS000102,IM0000944,Closed,2,2,2,0.79844416,...,,01-10-2013 18:20,01-10-2013 18:23,"2,43,70,27,778",User error,1,SD0001198,,1,C00000517
1126,WBA000124,application,Web Based Application,WBS000088,IM0001225,Closed,2,2,2,0.509362946,...,,,01-10-2013 15:34,"7,27,05,55,556",Other,1,SD0001639,,,
1128,OVR000033,,,WBS000118,IM0001227,Closed,2,2,2,0.647833302,...,,,01-10-2013 09:14,"0,807777778",Operator error,1,SD0001642,,1,C00001549
1129,WBA000124,application,Web Based Application,WBS000088,IM0001228,Closed,2,2,2,0.734261312,...,,,01-10-2013 15:34,"7,11,27,77,778",Other,1,SD0001641,,,
1185,CBD000911,computer,Banking Device,WBS000146,IM0001285,Closed,2,2,2,0.570666899,...,01-10-2013 12:56,01-10-2013 12:57,01-10-2013 12:57,"2,77,52,77,778",Other,1,SD0001738,,,
1186,CBD000910,computer,Banking Device,WBS000146,IM0001286,Closed,2,2,2,0.463489464,...,,01-10-2013 12:46,01-10-2013 12:46,"2,59,77,77,778",Hardware,1,SD0001741,,,
1203,WBA000148,application,Web Based Application,WBS000172,IM0001303,Closed,2,2,2,0.162128992,...,,01-10-2013 09:20,01-10-2013 09:21,"0,641111111",Other,1,SD0001584,,,
1221,CBD000666,computer,Banking Device,WBS000146,IM0001321,Closed,2,2,2,0.481147028,...,,01-10-2013 12:30,01-10-2013 12:30,"2,72,44,44,444",Hardware,1,SD0001855,,,
1244,CBD000419,computer,Banking Device,WBS000146,IM0001344,Closed,2,2,2,0.300271356,...,,03-10-2013 10:11,03-10-2013 10:11,"4,88,96,38,889",Hardware,1,SD0001775,,,
1315,CBD000884,computer,Banking Device,WBS000146,IM0001415,Closed,2,2,2,0.951568339,...,,01-10-2013 13:15,01-10-2013 13:15,"2,73,38,88,889",Hardware,1,SD0001900,,,


In [11]:
data.loc[data.Priority=='3'] ## 5323

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3,0.415049969,...,02-12-2013 12:31,02-12-2013 12:36,02-12-2013 12:36,"4,35,47,86,389",Software,1,SD0000011,1,,
14,SBA000698,application,Server Based Application,WBS000090,IM0000023,Closed,3,3,3,0.025310167,...,,12-12-2013 10:23,12-12-2013 10:24,"3,11,43,58,056",Data,1,SD0000045,,,
18,DTA000616,application,Desktop Application,WBS000092,IM0000033,Closed,3,3,3,0.860254961,...,,30-11-2013 10:39,30-11-2013 10:39,"2,62,67,33,889",Other,2,#MULTIVALUE,,,
38,WBA000124,application,Web Based Application,WBS000088,IM0000064,Closed,3,3,3,0.481872792,...,,08-11-2013 15:35,08-11-2013 15:35,"2,16,22,22,222",Software,1,SD0000090,,2,#MULTIVALUE
43,SUB000508,subapplication,Web Based Application,WBS000162,IM0000074,Closed,3,3,3,0.695187047,...,24-03-2014 12:35,24-03-2014 13:28,24-03-2014 13:28,"1,56,41,90,278",Other,2,#MULTIVALUE,1,1,C00000308
47,SBA000462,application,Server Based Application,WBS000073,IM0000081,Closed,4,3,3,0.461804336,...,,30-12-2013 10:39,30-12-2013 10:40,"1,92,41,11,111",Other,1,SD0000105,,,
51,WBA000124,application,Web Based Application,WBS000088,IM0000085,Closed,3,4,3,0.462625534,...,,08-11-2013 15:46,08-11-2013 15:46,"1,57,55,43,889",No error - works as designed,1,SD0000109,,,
60,SBA000782,application,Server Based Application,WBS000043,IM0000099,Closed,3,3,3,0.637701727,...,,09-12-2013 14:32,09-12-2013 14:32,"1,68,42,07,778",Other,1,SD0000125,,,
74,WBA000124,application,Web Based Application,WBS000088,IM0000115,Closed,3,3,3,0.232393197,...,,12-11-2013 13:38,12-11-2013 13:39,"3,37,09,72,222",Other,1,SD0000145,,,
76,SBA000765,application,Server Based Application,WBS000300,IM0000120,Closed,3,3,3,0.231178197,...,22-05-2013 15:27,,17-01-2014 15:00,"1,77,59,11,667",Unknown,1,SD0000149,,,


In [12]:
data.loc[data.Priority=='4'] ## 22717 incidents

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4,0.601292279,...,,04-11-2013 13:50,04-11-2013 13:51,"3,87,16,91,111",Other,1,SD0000007,2,,
3,WBA000124,application,Web Based Application,WBS000088,IM0000011,Closed,4,4,4,0.642927218,...,,14-11-2013 09:31,14-11-2013 09:31,"4,32,18,33,333",Operator error,1,SD0000025,,,
4,WBA000124,application,Web Based Application,WBS000088,IM0000012,Closed,4,4,4,0.345258343,...,,08-11-2013 13:55,08-11-2013 13:55,"3,38,39,03,333",Other,1,SD0000029,,,
5,WBA000124,application,Web Based Application,WBS000088,IM0000013,Closed,4,4,4,0.006675648,...,,08-11-2013 13:54,08-11-2013 13:54,"3,38,34,36,944",Other,1,SD0000031,,,
6,WBA000082,application,Web Based Application,WBS000055,IM0000014,Closed,4,4,4,0.19290712,...,,27-12-2013 10:59,27-12-2013 10:59,"3,70,31,91,389",Unknown,1,SD0000033,,,
7,WBA000124,application,Web Based Application,WBS000088,IM0000015,Closed,4,4,4,0.996467545,...,,08-11-2013 14:09,08-11-2013 14:09,"3,29,46,24,167",Other,1,SD0000034,,,
9,WBA000082,application,Web Based Application,WBS000055,IM0000018,Closed,4,4,4,0.306670312,...,,08-11-2013 14:33,08-11-2013 14:35,"3,21,15,26,944",No error - works as designed,1,SD0000037,,,
10,WBA000124,application,Web Based Application,WBS000088,IM0000019,Closed,4,4,4,0.439908807,...,,08-11-2013 14:23,08-11-2013 14:23,"3,06,74,49,167",Software,1,SD0000040,,1,C00000056
11,WBA000124,application,Web Based Application,WBS000088,IM0000020,Closed,4,4,4,0.291927638,...,,08-11-2013 14:18,08-11-2013 14:22,"1,32,26,19,167",Software,1,SD0000042,,,
12,WBA000082,application,Web Based Application,WBS000055,IM0000021,Closed,4,4,4,0.776486429,...,28-01-2014 14:07,04-02-2014 09:38,04-02-2014 09:38,"1,13,24,28,333",Software,2,#MULTIVALUE,,,


In [13]:
data.loc[data.Priority=='5'] ## 16486 incidents

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
19,WBA000082,application,Web Based Application,WBS000055,IM0000034,Closed,5,5,5,0.663615311,...,11-02-2014 14:50,28-03-2014 14:20,28-03-2014 14:20,"9,31,78,08,333",Other,2,#MULTIVALUE,,,
27,SBA000662,application,Server Based Application,WBS000071,IM0000048,Closed,5,5,5,0.748361437,...,17-10-2013 12:22,17-10-2013 14:00,17-10-2013 14:00,"3,88,55,55,556",No error - works as designed,2,#MULTIVALUE,,,
28,SBA000607,application,Server Based Application,WBS000263,IM0000049,Closed,5,5,5,0.246183023,...,,17-10-2013 15:40,17-10-2013 15:40,"1,85,28,81,944",Other,1,SD0000076,,,
29,SBA000263,application,Server Based Application,WBS000072,IM0000053,Closed,5,5,5,0.85372157,...,04-12-2013 13:58,,04-12-2013 14:00,"2,77,45,55,556",Other,2,#MULTIVALUE,,,
36,SUB000431,subapplication,Web Based Application,WBS000066,IM0000061,Closed,5,5,5,0.433426008,...,24-01-2014 15:20,,27-01-2014 12:44,"23,33",Software,2,#MULTIVALUE,,,
39,SBA000314,application,Server Based Application,WBS000054,IM0000067,Closed,5,5,5,0.719922533,...,08-10-2013 09:14,09-10-2013 13:41,09-10-2013 13:41,"8,73,98,05,556",Other,2,#MULTIVALUE,,,
42,SAP000005,application,SAP,WBS000271,IM0000073,Closed,5,5,5,0.683291495,...,,01-10-2013 07:52,01-10-2013 07:52,"6,83,96,30,556",Other,1,SD0000099,,,
45,SAP000004,application,SAP,WBS000271,IM0000078,Closed,5,5,5,0.775691867,...,,17-10-2013 11:43,17-10-2013 11:43,"1,45,90,33,333",Software,1,SD0000103,,,
52,SAP000004,application,SAP,WBS000271,IM0000088,Closed,5,5,5,0.773604774,...,12-11-2013 12:04,12-11-2013 12:13,12-11-2013 12:13,"7,06,115",Other,2,#MULTIVALUE,1,,
54,SBA000045,application,Server Based Application,WBS000170,IM0000090,Closed,5,5,5,0.283634692,...,,22-01-2014 16:19,22-01-2014 16:20,"21,97",Unknown,1,SD0000113,1,1,C00000582


In [14]:
data[data.Priority == 'NA'] ## 1380 incidents

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NA,0.517551335,...,,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1,SD0000017,,,
8,WBA000124,application,Web Based Application,WBS000088,IM0000017,Closed,NS,3,NA,0.269573825,...,,08-11-2013 14:02,08-11-2013 14:02,"0,862777778",Other,1,SD0000036,,,
103,SBA000782,application,Server Based Application,WBS000043,IM0000151,Closed,NS,3,NA,0.610829367,...,,17-10-2013 15:03,17-10-2013 15:03,"9,64,76,66,667",Other,1,SD0000177,,,
147,SBA000782,application,Server Based Application,WBS000043,IM0000199,Closed,NS,3,NA,0.724279149,...,,09-12-2013 14:49,09-12-2013 14:50,"11,26,525",Other,1,SD0000228,,,
152,SBA000782,application,Server Based Application,WBS000043,IM0000205,Closed,NS,3,NA,0.374449123,...,,09-12-2013 14:46,09-12-2013 14:46,"1,11,86,48,333",Other,1,SD0000233,,,
166,SBA000782,application,Server Based Application,WBS000043,IM0000219,Closed,NS,3,NA,0.93840208,...,,09-12-2013 14:27,09-12-2013 14:27,"1,09,02,19,444",Other,1,SD0000254,,,
183,SBA000607,application,Server Based Application,WBS000263,IM0000237,Closed,NS,3,NA,0.230755258,...,09-09-2013 09:40,08-10-2013 14:46,08-10-2013 14:47,"3,07,33,05,556",Other,4,#MULTIVALUE,,,
226,SBA000782,application,Server Based Application,WBS000043,IM0000287,Closed,NS,3,NA,0.283418566,...,,03-10-2013 11:18,03-10-2013 11:18,"3,95,45,61,111",Other,1,SD0000356,,,
237,STA000033,application,Standard Application,WBS000066,IM0000299,Closed,NS,3,NA,0.110482503,...,,18-11-2013 09:19,18-11-2013 09:19,"6,04,43,38,889",Software,1,SD0000456,,1,C00006478
277,SBA000759,application,Server Based Application,WBS000296,IM0000344,Closed,NS,3,NA,0.469216014,...,28-08-2013 15:38,13-11-2013 10:50,13-11-2013 10:50,"3,05,38,88,889",Software,2,#MULTIVALUE,1,,


In [15]:
## Converting NS and NA to NaN
data.Impact[data.Impact == 'NS'] = np.nan
data.Priority[data.Priority == 'NA'] = np.nan
data.isnull().values.any()

True

In [16]:
## Converting NaN by FFill
data = data.fillna(method='ffill')

In [17]:
data.columns

Index(['CI_Name', 'CI_Cat', 'CI_Subcat', 'WBS', 'Incident_ID', 'Status',
       'Impact', 'Urgency', 'Priority', 'number_cnt', 'Category', 'KB_number',
       'Alert_Status', 'No_of_Reassignments', 'Open_Time', 'Reopen_Time',
       'Resolved_Time', 'Close_Time', 'Handle_Time_hrs', 'Closure_Code',
       'No_of_Related_Interactions', 'Related_Interaction',
       'No_of_Related_Incidents', 'No_of_Related_Changes', 'Related_Change'],
      dtype='object')

In [18]:
## Label Encoding all the elements
enc = LabelEncoder()
data.CI_Name = enc.fit_transform(data.CI_Name)
data.CI_Cat = enc.fit_transform(data.CI_Cat)
data.CI_Subcat = enc.fit_transform(data.CI_Subcat)
data.WBS = enc.fit_transform(data.WBS)
data.Incident_ID = enc.fit_transform(data.Incident_ID)
data.Status = enc.fit_transform(data.Status)
data.Impact = enc.fit_transform(data.Impact)
data.Urgency = enc.fit_transform(data.Urgency)
data.Priority = enc.fit_transform(data.Priority)
data.number_cnt = enc.fit_transform(data.number_cnt)
data.Category = enc.fit_transform(data.Category)
data.KB_number = enc.fit_transform(data.KB_number)
data.Alert_Status = enc.fit_transform(data.Alert_Status)
data.No_of_Reassignments = enc.fit_transform(data.No_of_Reassignments)
data.Open_Time = enc.fit_transform(data.Open_Time)
data.Reopen_Time = enc.fit_transform(data.Reopen_Time)
data.Resolved_Time = enc.fit_transform(data.Resolved_Time)
data.Close_Time = enc.fit_transform(data.Close_Time)
data.Handle_Time_hrs = enc.fit_transform(data.Handle_Time_hrs)
data.Closure_Code = enc.fit_transform(data.Closure_Code)
data.No_of_Related_Interactions = enc.fit_transform(data.No_of_Related_Interactions)
data.Related_Interaction = enc.fit_transform(data.Related_Interaction)
data.No_of_Related_Incidents = enc.fit_transform(data.No_of_Related_Incidents)
data.No_of_Related_Changes = enc.fit_transform(data.No_of_Related_Changes)
data.Related_Change = enc.fit_transform(data.Related_Change)

In [19]:
data.head()

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,2741,12,58,137,0,0,3,3,3,28044,...,0,3526,3659,19911,7,1,2,10,0,0
1,2863,2,58,70,1,0,2,2,2,19341,...,76,1181,1222,21448,11,1,3,1,0,0
2,990,2,11,74,2,0,2,2,2,24089,...,0,13274,13734,22732,5,1,4,0,0,0
3,2863,2,58,70,3,0,3,3,3,29998,...,0,15382,15862,21363,6,1,5,0,0,0
4,2863,2,58,70,4,0,3,3,3,16110,...,0,8422,8699,18369,7,1,6,0,0,0


In [25]:
data.corr(method='pearson',min_periods=1)

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
CI_Name,1.000000,0.168426,0.949575,-0.106713,0.019721,-0.034878,0.098902,0.120105,0.105974,-0.004649,...,0.010211,0.059412,0.010631,-0.043334,0.122666,-0.018304,0.025931,0.012597,0.007194,-0.026561
CI_Cat,0.168426,1.000000,0.204052,-0.217604,0.032207,-0.000471,-0.054472,-0.048293,-0.052406,0.002270,...,-0.017528,0.017165,0.008856,-0.069631,-0.039637,-0.022926,0.035069,-0.018819,0.002674,-0.004059
CI_Subcat,0.949575,0.204052,1.000000,-0.122366,0.033585,-0.032120,0.069299,0.094432,0.078041,-0.004755,...,0.013067,0.053395,0.010394,-0.050114,0.150923,-0.016023,0.037078,0.011213,0.009454,-0.022619
WBS,-0.106713,-0.217604,-0.122366,1.000000,0.003462,0.003152,0.036365,0.023091,0.032962,0.007228,...,0.003668,0.006180,0.006468,0.076104,-0.069614,-0.008521,0.009129,0.020009,-0.014900,0.000461
Incident_ID,0.019721,0.032207,0.033585,0.003462,1.000000,-0.017455,-0.029767,-0.033627,-0.029251,0.006732,...,-0.028250,0.167413,0.180279,-0.070408,0.012528,-0.045406,0.885116,0.013074,-0.021638,0.031455
Status,-0.034878,-0.000471,-0.032120,0.003152,-0.017455,1.000000,-0.019650,-0.022810,-0.019490,0.006356,...,-0.002717,0.007182,0.006561,0.001177,-0.006225,-0.003534,-0.014834,-0.001130,-0.001489,-0.001290
Impact,0.098902,-0.054472,0.069299,0.036365,-0.029767,-0.019650,1.000000,0.929447,0.991623,0.001541,...,0.037462,0.062167,0.003485,0.090121,-0.043543,-0.006255,-0.027491,-0.064046,-0.043357,-0.046081
Urgency,0.120105,-0.048293,0.094432,0.023091,-0.033627,-0.022810,0.929447,1.000000,0.939132,0.000798,...,0.038751,0.064262,0.004025,0.100362,-0.047087,-0.004972,-0.030847,-0.063526,-0.046364,-0.052578
Priority,0.105974,-0.052406,0.078041,0.032962,-0.029251,-0.019490,0.991623,0.939132,1.000000,0.000928,...,0.037387,0.062610,0.003538,0.089226,-0.043887,-0.007346,-0.026526,-0.064087,-0.043685,-0.046407
number_cnt,-0.004649,0.002270,-0.004755,0.007228,0.006732,0.006356,0.001541,0.000798,0.000928,1.000000,...,0.008046,0.004522,0.006213,-0.001313,-0.000741,0.006087,0.004184,0.005581,0.001775,0.002232


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46606 entries, 0 to 46605
Data columns (total 25 columns):
CI_Name                       46606 non-null int64
CI_Cat                        46606 non-null int64
CI_Subcat                     46606 non-null int64
WBS                           46606 non-null int64
Incident_ID                   46606 non-null int64
Status                        46606 non-null int64
Impact                        46606 non-null int64
Urgency                       46606 non-null int64
Priority                      46606 non-null int64
number_cnt                    46606 non-null int64
Category                      46606 non-null int64
KB_number                     46606 non-null int64
Alert_Status                  46606 non-null int64
No_of_Reassignments           46606 non-null int64
Open_Time                     46606 non-null int64
Reopen_Time                   46606 non-null int64
Resolved_Time                 46606 non-null int64
Close_Time              

In [21]:
X = data.loc[:,['Impact','Status','No_of_Related_Incidents','Urgency']]
y = data.Priority
print(X.shape)
print(y.shape)

(46606, 4)
(46606,)


### Linear Regression

In [22]:
## Splitting the data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.35,random_state = 10)
model_LR = LinearRegression()
model_LR.fit(X_train,y_train)
predict_LR = model_LR.predict(X_test)

In [23]:
r2_score(y_test,predict_LR)

0.9848476607051311

In [26]:
predict_LR

array([3.99414554, 2.87858199, 1.99866806, ..., 3.99414554, 2.9964068 ,
       2.9964068 ])

In [77]:
df = pd.DataFrame([[1,0,2,15]])

In [78]:
predict2 = model_LR.predict(df)
predict2

array([2.65024573])

In [92]:
## Implement a function to the model to predict priorties
def Linear_regression(Dataframe):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.35,random_state = 10)
    model_LR = LinearRegression()
    model_LR.fit(X_train,y_train)
    predict_LR = model_LR.predict(Dataframe)
    return predict_LR

In [93]:
df = pd.DataFrame([[1,0,2,15]])
Linear = Linear_regression(df)

In [94]:
Linear

array([2.65024573])